<h1 align=center> Tarea 7: Decision Trees y Random Forest
 </h1>

Integrantes:
- Axel Mondaca
- Sebastián Hernández

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, recall_score, accuracy_score

